In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U transformers
!pip install -q -U accelarate
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U peft

In [ ]:
import os
import warnings
import pandas as pd 
import numpy as np 
import torch 
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer, setup_chat_format

In [ ]:
data = pd.read_csv('/kaggle/input/ecommerce-llm-dataset/Merged_Ecommerce_FAQs.csv')
data.head(10)

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
dataset_path = "/kaggle/input/ecommerce-llm-dataset"

In [ ]:
from datasets import Dataset
train_data = Dataset.from_pandas(data[['prompt', 'response']])
print(train_data)

In [ ]:
compute_dtype = getattr(torch, 'float16')

bnbconfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnbconfig, 
    torch_dtype=compute_dtype, 
    low_cpu_mem_usage=True,
    #trust_remote_code=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model, tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["prompt"]},
               {"role": "assistant", "content": row["response"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = train_data.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

In [ ]:
training_arguments = TrainingArguments(
    output_dir="banking_chatbot_specialist",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
)

In [ ]:
from peft import get_peft_model, prepare_model_for_kbit_training, PeftModel
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#model.resize_token_embeddings(len(tokenizer))  # Update model's embeddings to include the new token

In [ ]:
trainer = SFTTrainer(
    model = model, 
    train_dataset = dataset, 
    peft_config = peft_config, 
    tokenizer = tokenizer, 
    args = training_arguments,  
)

trainer.train()

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Hi, how can i track my order?\n"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[0])

In [ ]:
model_dir = "Phi-3.5-mini-instruct-Ecommerce-Text-Classification"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)